# Parameter example

## 1. Setup

 Setup notebook:

In [15]:
# just for the notebook setup so we have the ami_sim module
import os
import sys
sys.path.append('../')
os.environ['SOSSDIR'] = "/data/jwst_soss/data/neil_wrap"

First we import from soss_sim

In [16]:
from soss_mtl.core.instrument import constants
from soss_mtl.core.core import param_functions

All codes should have the following variables (at the start of the code)

In [17]:
# =============================================================================
# Define variables
# =============================================================================
# set name
__NAME__ = 'bin/test.py'
__DESCRIPTION__ = 'test script for SOSS_SIM_MTL'
# get default constants
consts = constants.Consts
# copy for update
lconsts = consts.copy(__NAME__)
# set very basic constants
__VERSION__ = lconsts.constants['PACKAGE_VERSION'].value
__DATE__ = lconsts.constants['PACKAGE_VERSION_DATE'].value

Define custom arguments (just for this code)

Example here we add a "scene" argument with the following properties:
1. name = "SCENE"          # parameter name = "SCENE"
2. dtype = str             # data type = python string
3. source = `__NAME__`     # where the parameter was defined (here `__NAME__` is the code name)
4. user = True             # whether to populate this value in any config file generated
5. argument = True         # whether to use this variable from command line (must define "command" in this case)
6. group = 'code'          # which group this variable belongs to (mostly for config file)
7. description = ...       # a description for the help file / config file
8. command = `['--scene']` # a list of commands to be able to use at run time e.g. in this case --scene=test.fits

In [18]:
# Define the scene fits file
lconsts.add_argument('TESTVALUE', value=None, dtype=str,
                     source=__NAME__, user=True, argument=True,
                     group='code', description='Define a test value',
                     command=['--value'])

Now we can get parameters:

In [19]:
kwargs = dict()                   # this is used for function calls (ignore for now)
sys.argv = 'test.py'.split()      # this is where command lines arguments come from
params = param_functions.setup(lconsts, kwargs, desc=__DESCRIPTION__)

*******************************************************************************
 soss_mtl        Version: 0.0.001
*******************************************************************************


For example params currently has the following:

In [20]:
params['ENV_DIR']
os.environ[params['ENV_DIR']]

'/data/jwst_soss/data/neil_wrap'

In [21]:
params

ParamDict:

 	DEBUG:                        0                                             # core.instruments.constants.py
 	USER_CONFIG_FILE:             None                                          # core.instruments.constants.py
 	PACKAGE_NAME:                 soss_mtl                                      # core.instruments.constants.py
 	PACKAGE_VERSION:              0.0.001                                       # core.instruments.constants.py
 	PACKAGE_VERSION_DATE:         2021-05-27                                    # core.instruments.constants.py
 	PACKAGE_THEME:                DARK                                          # core.instruments.constants.py
 	PACKAGE_DIRECTORY:            soss_sim_data                                 # core.instruments.constants.py
 	GENERATE_CONFIG_FILE:         False                                         # core.instruments.constants.py
 	ENV_DIR:                      SOSSDIR                                       # core.instruments.constants.p

Let test this again with some arguments from the command line as follows:

In [22]:
sys.argv = 'test.py --value=test.fits'.split()

In [23]:
kwargs = dict()                   # this is used for function calls
params = param_functions.setup(lconsts, kwargs, desc=__DESCRIPTION__)

*******************************************************************************
 soss_mtl        Version: 0.0.001
*******************************************************************************


In [24]:
params

ParamDict:

 	DEBUG:                        0                                             # core.instruments.constants.py
 	USER_CONFIG_FILE:             None                                          # core.instruments.constants.py
 	PACKAGE_NAME:                 soss_mtl                                      # core.instruments.constants.py
 	PACKAGE_VERSION:              0.0.001                                       # core.instruments.constants.py
 	PACKAGE_VERSION_DATE:         2021-05-27                                    # core.instruments.constants.py
 	PACKAGE_THEME:                DARK                                          # core.instruments.constants.py
 	PACKAGE_DIRECTORY:            soss_sim_data                                 # core.instruments.constants.py
 	GENERATE_CONFIG_FILE:         False                                         # core.instruments.constants.py
 	ENV_DIR:                      SOSSDIR                                       # core.instruments.constants.p

Note the difference between the value of `ADD_JITTER` and `SCENE` from the previous example, the source (third column above has also changed -- to `sys.argv`)

## 2. Generating a config file

There is a special mode for any code using the param_fucntions.setup function.

One can generate a config file with the current settings given (from command line / another config file / the constants file)

This is done using the argument `--getconfig=True` (the default value is `--getconfig=False`)

In [25]:
# set the command line arguments (for notebook only)
sys.argv = 'test.py --getconfig True'.split()

In [26]:
kwargs = dict()                   # this is used for function calls

try:
    params = param_functions.setup(lconsts, kwargs, desc=__DESCRIPTION__)
# just so the notebook continues after this
except Exception as e:
    print('Error {0}: {1}'.format(e.__name__, e))

*******************************************************************************
 soss_mtl        Version: 0.0.001
*******************************************************************************
2021-05-27 14:24:53,633 | INFO  | Writing constants file to config/None


As you can see above this added a file to `./outputs/` called `user_config.ini`

This file can be called from the command line using the `--config` argument.

## 3. The help file

As mentioned above the descriptions we gave of `SCENE` will appear in the help file.

The help file is accessed as always through `-h` or `--help`. 

We can emulate this here with `sys.argv`:

In [27]:
# set the command line arguments (for notebook only)
sys.argv = 'test.py --help'.split()

which will run when we run the `param_functions.setup` function:

In [28]:
kwargs = dict()                   # this is used for function calls
# ignore the try/except statement here it is just because the -h/--help argument will force an exit of python
try:
    params = param_functions.setup(lconsts, kwargs, desc=__DESCRIPTION__)
except SystemExit:
    pass

usage: test.py [-h] [--debug DEBUG] [--uconfig USER_CONFIG_FILE]
               [--theme PACKAGE_THEME] [--getconfig GENERATE_CONFIG_FILE]
               [--dir DIRECTORY] [--value TESTVALUE]

test script for SOSS_SIM_MTL

optional arguments:
  -h, --help            show this help message and exit
  --debug DEBUG         Set debug mode (1-9) the higher the number the more
                        verbose
  --uconfig USER_CONFIG_FILE
                        Define the user config file
  --theme PACKAGE_THEME
                        Define package log theme
  --getconfig GENERATE_CONFIG_FILE
                        Define whether we want to generate a config file
  --dir DIRECTORY       Define the working directory (note this can also be
                        set by setting the SOSSDIR environmental variable)
  --value TESTVALUE     Define a test value


## 4. Use in a code

We put the main code we with to run in a `__main__()` sub function so we can log and manage exception that come from the code. When we call the code we use the `main()` function that will run the parameter setup and handle any errors from our `__main__()` function. (By handle here I mean deal with exceptions and log/shut things done in a good way).

A code using a good setup would look as follows:

In [29]:
def main(**kwargs):
    # get params (run time + config file + constants file)
    params = param_functions.setup(lconsts, kwargs, desc=__DESCRIPTION__)
    # run the __main__ to return products
    if not params['GENERATE_CONFIG_FILE']:
        # note eventually this will be a call to a function which manages exceptions
        return __main__(params)


def __main__(params):
    # main code here
    print('Hello, World')
    print('My code goes here')
    return params
    
    

This is then called from the main code (or from an import) as follows:

In [30]:
# For the note book we need to set the command line arguments
sys.argv = 'test.py'.split()
# run main code
ll = main()


*******************************************************************************
 soss_mtl        Version: 0.0.001
*******************************************************************************


Hello, World
My code goes here


Note to import this it would look as follows:


```
import paramtest

test.main()
```


where any parameters can be defined in the `.main()` call i.e.:

In [31]:
# just for the notebook
sys.path.append('../bin/')
# import code
import paramtest

# the parameter test function just prints out information about the parameters
paramtest.main(add_jitter=False, scene='loicfile.fits')

*******************************************************************************
 soss_mtl        Version: 0.0.001
*******************************************************************************
 Code: paramtest
*******************************************************************************
2021-05-27 14:24:53,666 | INFO  | This is a test of info
2021-05-27 14:24:53,666 | WARNI | This is a test of warning
2021-05-27 14:24:53,668 | ERROR | This is a test of an error


Information for key = 'DEBUG'
	Data Type: 		 int
	Value: 		 	0                                            
	Source: 		 core.instruments.constants.py
	Instance: 		 Constant[DEBUG]
Information for key = 'USER_CONFIG_FILE'
	Data Type: 		 NoneType
	Value: 		 	None                                         
	Source: 		 core.instruments.constants.py
	Instance: 		 Constant[USER_CONFIG_FILE]
Information for key = 'PACKAGE_NAME'
	Data Type: 		 str
	Value: 		 	soss_mtl                                     
	Source: 		 core.instruments.constants.py
	Instance: 		 Constant[PACKAGE_NAME]
Information for key = 'PACKAGE_VERSION'
	Data Type: 		 str
	Value: 		 	0.0.001                                      
	Source: 		 core.instruments.constants.py
	Instance: 		 Constant[PACKAGE_VERSION]
Information for key = 'PACKAGE_VERSION_DATE'
	Data Type: 		 str
	Value: 		 	2021-05-27                                   
	Source: 		 core.instruments.constants.py
	Instance: 		 Constant[PACKAGE_VERSION_DATE]
Information fo

ParamDict:

 	DEBUG:                        0                                             # core.instruments.constants.py
 	USER_CONFIG_FILE:             None                                          # core.instruments.constants.py
 	PACKAGE_NAME:                 soss_mtl                                      # core.instruments.constants.py
 	PACKAGE_VERSION:              0.0.001                                       # core.instruments.constants.py
 	PACKAGE_VERSION_DATE:         2021-05-27                                    # core.instruments.constants.py
 	PACKAGE_THEME:                DARK                                          # core.instruments.constants.py
 	PACKAGE_DIRECTORY:            soss_sim_data                                 # core.instruments.constants.py
 	GENERATE_CONFIG_FILE:         False                                         # core.instruments.constants.py
 	ENV_DIR:                      SOSSDIR                                       # core.instruments.constants.p

Note again the values of `add_jitter` and `scene` and the source location (`kwargs`)

## 5. The parameter dictionary (ParamDict)

The paremeter dictionary has multiple useful features.

### 5.1 Parameter dictionary is case-insensitive

This means unlike a normal dictionary you only have one value describing and characters i.e.:

- PACKAGE_VERSION
- Package_Version
- package_version
- PackAge_VerSion

all link to the same constant:

In [32]:
print('version = ', params['PACKAGE_VERSION'])
print('version = ', params['Package_Version'])
print('version = ', params['package_version'])
print('version = ', params['PackAge_VerSion'])

version =  0.0.001
version =  0.0.001
version =  0.0.001
version =  0.0.001


### 5.2 Parameter dictionary is locked

In most cases the parameter dictionary `params` should not be added to or modified after the `setup` function. To aid this there is a locking mechanism that will prevent adding to or changing the parameter dictionary:

In [33]:
params['TEST'] = 2
print('TEST = ', params['TEST'])

TEST =  2


In [34]:
params.lock()

In [35]:
params['TEST'] = 1
print('TEST = ', params['TEST'])

ParamDictException: [set] ParamDict locked. 
	 Cannot add 'TEST'='1'
	Func: core.core.constant_functions.py.ParamDict.__setitem__()

However in exceptional circumstances it is possible:

In [36]:
params.set('TEST', value=3, source=__NAME__)
print('TEST = ', params['TEST'])

TEST =  3


### 5.3 Find out where a parameter was defined:

In [37]:
params.sources['SCENE']

KeyError: 'SCENE'

In [38]:
params.sources['PACKAGE_VERSION']

'core.instruments.constants.py'

### 5.4 Get information about parameters:

In [39]:
params.info('SCENE')

ParamDict Info: Key = 'SCENE' not found


In [40]:
params.info('JITTER_RMS')

ParamDict Info: Key = 'JITTER_RMS' not found


In [41]:
import numpy as np
params.set('LOICARRAY', np.arange(100), source='test')

params.info('LOICARRAY')


Information for key = 'LOICARRAY'
	Data Type: 		 ndarray
	Min Value: 		 0 
	Max Value: 		 99 
	 Has NaNs: 		 False 
	 Values: 		 ['\t[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...  ']
	Source: 		 test
	Instance: 		 None


### 5.5 Lists and dictionaries from strings

Because config files and arguments may have trouble with inputs such as list and dictionary the parameter file has special ways to open these, these methods are `listp` and `dictp`.

In [42]:
# lets unlock the parameter dictionary for now
params.unlock()

# lets define a parameter (that may come from the constants file, config file or command line)
# You'll notice the "list" here is not like python but just a python string separated by commas
params['WAVES'] = '400, 500, 600'

To use this 'string list' as a list we have the `listp` method:

In [43]:
wavelengths = params.listp('WAVES', dtype=float)
print(wavelengths)

[400.0, 500.0, 600.0]


The same is true for dictionaries (note be careful with the use of `'` and `"`)

In [44]:
params['COLOURS'] = "{'red': 'r', 'blue': 'b', 'green': 'g'}"
params['PEOPLE'] = '{"bob":10, "fred":20, "chris": 30}'

In [45]:
colours = params.dictp('COLOURS')
print(colours)

{'red': 'r', 'blue': 'b', 'green': 'g'}


In [46]:
people = params.dictp('PEOPLE')
print(people)

{'bob': 10, 'fred': 20, 'chris': 30}


In our constants file these would look as follows:

    # define the wavelengths
    WAVES = 400, 500, 600

    # define the colours for plotting
    COLOURS = {'red': 'r', 'blue': 'b', 'green': 'g'}

    # define the people
    PEOPLE = {"bob":10, "fred":20, "chris": 30}


### 5.6 Finding a parameter

There are some useful string methods that are also usable to search for specific keys (useful when there are many constants defined)

In [47]:
params.startswith('PACK')

['PACKAGE_NAME',
 'PACKAGE_VERSION',
 'PACKAGE_VERSION_DATE',
 'PACKAGE_THEME',
 'PACKAGE_DIRECTORY']

In [50]:
params.contains('NAME')

['PACKAGE_NAME']

In [51]:
params.endswith('DATE')

['PACKAGE_VERSION_DATE']

In [52]:
list(params.keys())

['DEBUG',
 'USER_CONFIG_FILE',
 'PACKAGE_NAME',
 'PACKAGE_VERSION',
 'PACKAGE_VERSION_DATE',
 'PACKAGE_THEME',
 'PACKAGE_DIRECTORY',
 'GENERATE_CONFIG_FILE',
 'ENV_DIR',
 'DIRECTORY',
 'TESTVALUE',
 'CONFIGDIR',
 'PID',
 'PIDTIME',
 'INPUTDIR',
 'OUTPUTDIR',
 'LOGDIR',
 'LOGFILE',
 'TEST',
 'LOICARRAY',
 'WAVES',
 'COLOURS',
 'PEOPLE']